In [ ]:
#Import dependencies

from numpythia import Pythia, STATUS, HAS_END_VERTEX, ABS_PDG_ID, hepmc_write, hepmc_read
from pyjet import cluster

%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import numpy as np
import pandas as pd

import ROOT

In [ ]:
#In selection do not allow it to be a neutrino

selection = ((STATUS == 1) & ~HAS_END_VERTEX & (ABS_PDG_ID != 12) & (ABS_PDG_ID != 14) & (ABS_PDG_ID != 16))

In [ ]:
#Photoproduction (low Q2)

params_pp = {
 "HardQCD:all":          "on",  #hard subprocesses (2->2) are considered
 "Beams:eCM":            100.,  #center of mass energy
 "Beams:idA":            11,    #identity of incoming object A in the collision (e)
 "Beams:idB":            2212,  #identity of incoming object B in the collision (p)
 "PDF:lepton2gamma":     "on",  #initial state radiation for leptons 
 "Photon:Q2max":         1.,    #upper limit for real photon virtuality (can only handle (quasi-)real photons)
 "Photon:Wmin":          10.,   #minimum for the invariant mass (total energy and momentum) of a photon-hadron system
 "PhaseSpace:pTHatMin":  2.,    #minimum pT 
 "PhotonParton:all":     "on",  #photon-parton interactions (gmg->qqbar,ccbar,ddbar)
 "Photon:ProcessType":   0,     #mixed contributions from resolved and direct photon interactions (resolved=hadronic photon behavior)
 "Main:timesAllowErrors":10000, #allowed number of errors before aborting
}

In [ ]:
#DIS process

params_dis = {
 "Beams:frameType":                2,    #set-up frame for beams with unequal energies
 "Beams:idA":                      2212, #identity of incoming object A in the collision (p)
 "Beams:idB":                      11,   #identity of incoming object B in the collision (e)
 "Beams:eA" :                      250., #beam energy of incoming object A in the collision
 "Beams:eB"  :                     20.,  #beam energy of incoming object B in the collision
    
 #Set up DIS process within some phase space.
    
 "WeakBosonExchange:ff2ff(t:gmZ)": "on", #turn on neutral currents (with gamma/Z interference)
 "PhaseSpace:Q2Min":               1.,   #minimum for real photon virtuality (phase space cut)
 "SpaceShower:pTmaxMatch":         2.,   #always allow emissions up to the kinematical limit
 "PDF:lepton":                     "off",#turn off QED radiation from lepton (lepton carries the full beam energy)
 "TimeShower:QEDshowerByL":        "off",#don't allow leptons to radiate photons
}

In [ ]:
pythia = Pythia(params = params_dis)

Output dtype = [(′E′,′f8′),(′px′,′f8′),(′py′,′f8′),(′pz′,′f8′),(′mass′,′f8′),(′prodx′,′f8′),(′prody′,′f8′),(′prodz′,′f8′),(′prodt′,′f8′),(′pdgid′,′i4′),(′status′,′i4′)]

In [ ]:
def getData(inputFile):
    
    j = 0
    k = 0
    l = 0
    m = 0
    n = 0
    p = 0
    q = 0
    t = 0
    z = 0

################################

    p_mass    = 0

    eIn_pt    = [0]*1000000
    eIn_p     = [0]*1000000

    eIn_4vec  = ROOT.TLorentzVector()

    p_pt      = [0]*1000000
    p_p       = [0]*1000000

    p_4vec    = ROOT.TLorentzVector()

    eOut_pt   = [0]*1000000
    eOut_p    = [0]*1000000
    eOut_eta  = [0]*1000000

    eOut_4vec = ROOT.TLorentzVector()

    Q2        = [0]*1000000
    W2        = [0]*1000000
    x_bjork   = [0]*1000000
    y         = [0]*1000000
    nu        = [0]*1000000

################################

    Q2_jets_antiKt = [0]*1000000
    Q2_jets_CamAac = [0]*1000000

################################

    jet_pt_antiKt   = [0]*1000000
    jet_eta_antiKt  = [0]*1000000
    jet_phi_antiKt  = [0]*1000000
    jet_mass_antiKt = [0]*1000000
    jet_mult_antiKt = [0]*1000000

    jet_pt_CamAac   = [0]*1000000
    jet_eta_CamAac  = [0]*1000000
    jet_phi_CamAac  = [0]*1000000
    jet_mass_CamAac = [0]*1000000
    jet_mult_CamAac = [0]*1000000

    phi_corr = [0]*1000000
    eta_corr = [0]*1000000

################################

    R_0 = [0]*1000000
    R_1 = [0]*1000000

    angularity_0 = [0]*1000000
    angularity_1 = [0]*1000000

################################

    for event in inputFile: 
    
        if not event: continue
    
        jets_antiKt = cluster(event.all(selection & (ABS_PDG_ID != 11)), R=1, p=-1, ep=True).inclusive_jets(ptmin=5)
        jets_CamAac = cluster(event.all(selection & (ABS_PDG_ID != 11)), R=1, p= 0, ep=True).inclusive_jets(ptmin=5)

        if(len(jets_antiKt)<1): continue

################################

        for i in event.all():
        
            if i['pdgid'] == 11 and i['status'] == 4:
            
                #mass      = ROOT.TDatabasePDG.Instance().GetParticle(11).Mass()
                eIn_pt[j] = np.sqrt(i['px']*i['px'] + i['py']*i['py'])
                eIn_p[j]  = np.sqrt(i['px']*i['px'] + i['py']*i['py'] + i['pz']*i['pz'])
            
                #make its respective 4 vector which can be used, along with the others, to find Q2, W, x_borj, and y

                eIn_4vec.SetPxPyPzE(i['px'], i['py'], i['pz'], i['E'])
            
                j = j  + 1
            
            elif i['pdgid'] == 2212 and i['status'] == 4:
            
                p_pt[k] = np.sqrt(i['px']*i['px'] + i['py']*i['py'])
                p_p[k]  = np.sqrt(i['px']*i['px'] + i['py']*i['py'] + i['pz']*i['pz'])
            
                p_mass  = i['mass']
            
                p_4vec.SetPxPyPzE(i['px'], i['py'], i['pz'], i['E'])
        
                k = k  + 1
            
            elif i['pdgid'] == 11 and i['status'] == 1 and i['prodt'] == 0:
            
                eOut_pt[l]  = np.sqrt(i['px']*i['px'] + i['py']*i['py'])
                eOut_p[l]   = np.sqrt(i['px']*i['px'] + i['py']*i['py'] + i['pz']*i['pz'])
                eOut_eta[l] = 0.5*np.log((eOut_p[l] + i['pz'])/(eOut_p[l] - i['pz']))
            
                eOut_4vec.SetPxPyPzE(i['px'], i['py'], i['pz'], i['E'])
            
                l = l  + 1
            
        def dot(a_4vec, b_4vec):
        
            num = a_4vec(3)*b_4vec(3) - a_4vec(0)*b_4vec(0) - a_4vec(1)*b_4vec(1) - a_4vec(2)*b_4vec(2)
        
            return num
        
        gm_4vec = eIn_4vec - eOut_4vec
    
        Q2[m]      = -gm_4vec.Mag2()
        W2[m]      = (p_4vec + gm_4vec).Mag2()
        x_bjork[m] = Q2[m] / (2*dot(p_4vec,gm_4vec))
        y[m]       = (dot(p_4vec,gm_4vec)) / (dot(p_4vec,eIn_4vec))
        nu[m]      = (dot(p_4vec,gm_4vec)) / (p_mass)
        
        m = m + 1

################################

        for jet in jets_antiKt:
        
            jet_pt_antiKt[n]   = jet.pt
            jet_eta_antiKt[n]  = jet.eta
            jet_phi_antiKt[n]  = jet.phi
            jet_mass_antiKt[n] = jet.mass
            jet_mult_antiKt[n] = len(jet)
        
            phi_corr[n] = np.abs(ROOT.TVector2.Phi_mpi_pi(jet.phi-gm_4vec.Phi()))
        
            eta_corr[n] = np.abs(jet.eta-gm_4vec.Eta())

            ###angularity
            for part in jet:

                R_0[p] += part.pt * ((ROOT.TVector2.Phi_mpi_pi(jet_phi_antiKt[n]-part.phi))**2 + (jet_eta_antiKt[n]-part.eta)**2)
                R_1[p] += part.pt * np.sqrt((ROOT.TVector2.Phi_mpi_pi(jet_phi_antiKt[n]-part.phi))**2 + (jet_eta_antiKt[n]-part.eta)**2)
            
            angularity_0[p] = (1 / jet_pt_antiKt[n]) * R_0[p]
            angularity_1[p] = (1 / jet_pt_antiKt[n]) * R_1[p]

            n = n + 1
            p = p + 1
        
################################
        
        r_1 = 0
        r_2 = 0
    
        while r_2 < (r_1 + len(jets_antiKt)):
            Q2_jets_antiKt[t] = Q2[m - 1]
            r_2 = r_2 + 1
            t = t + 1   
        
################################
        
        if(len(jets_CamAac)<1): continue
        
        for jet in jets_CamAac:
        
            jet_pt_CamAac[q]   = jet.pt
            jet_eta_CamAac[q]  = jet.eta
            jet_phi_CamAac[q]  = jet.phi
            jet_mass_CamAac[q] = jet.mass
            jet_mult_CamAac[q] = len(jet)
        
            q = q + 1
        
################################
        
        x_1 = 0
        x_2 = 0
    
        while x_2 < (x_1 + len(jets_CamAac)):
            Q2_jets_CamAac[z] = Q2[m - 1]
            x_2 = x_2 + 1
            z = z + 1   
        
################################
        
    part_dataframe        = pd.DataFrame({'eIn_pt': eIn_pt, 'eOut_pt': eOut_pt, 'eIn_p': eIn_p, 'eOut_p': eOut_p, 
                               'eOut_eta': eOut_eta, 'p_pt': p_pt, 'p_p': p_p, 'Q2': Q2, 'W2': W2, 'x_bjork': x_bjork,
                               'y': y, 'nu': nu, 'phi_corr': phi_corr , 'eta_corr': eta_corr})
    part_data             = part_dataframe.query('eIn_pt != 0 and p_pt != 0 and y < 0.95')

    jets_antiKt_dataframe = pd.DataFrame({'jet_pt_antiKt': jet_pt_antiKt, 'jet_eta_antiKt': jet_eta_antiKt, 
                                      'jet_phi_antiKt':jet_phi_antiKt,'jet_mass_antiKt': jet_mass_antiKt, 
                                      'jet_mult_antiKt':jet_mult_antiKt,'angularity_0': angularity_0,   
                                      'angularity_1': angularity_1, 'Q2': Q2_jets_antiKt})
    jets_antiKt_data      = jets_antiKt_dataframe.query('jet_pt_antiKt != 0')

    jets_CamAac_dataframe = pd.DataFrame({'jet_pt_CamAac': jet_pt_CamAac,  'jet_eta_CamAac':jet_eta_CamAac,
                                      'jet_phi_CamAac': jet_phi_CamAac,'jet_mass_CamAac':jet_mass_CamAac,
                                      'jet_mult_CamAac': jet_mult_CamAac, 'Q2': Q2_jets_CamAac})
    jets_CamAac_data      = jets_CamAac_dataframe.query('jet_pt_CamAac != 0')

    return part_data, jets_antiKt_data, jets_CamAac_data

In [ ]:
#I generate my own data using pythia(events=numEvents)

numEvents = 3e4
part_data, jets_antiKt_data, jets_CamAac_data = getData(pythia(events=numEvents))

Q1 = part_data.query('Q2 > 1 and 10 >  Q2')
Q2 = part_data.query('Q2 > 10')

#Or I can read hepmc formatted data using hepmc_read('/Users/jose/...')

part_data_com, jets_antiKt_data_com, jets_CamAac_data_com = getData(hepmc_read('/Users/jose/Dropbox/for_Jose/lowQ2_eic_general.hepmc'))

In [ ]:
jets_pt_1 = np.array(['5','6','7','8','9','10','11','12','13','14','15'])
jets_pt_2 = np.array(['15','16','17','18','19','20','21','22','23','24','25'])
jets_pt_3 = np.array(['25','26','27','28','29','30'])

jet_mult_x     = np.array([5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5,
                           20.5, 21.5, 22.5, 23.5, 24.5, 25.5, 26.5, 27.5, 28.5, 29.5])
jet_mult_x_err = [0.5]*(jet_mult_x.shape[0])

In [ ]:
def bins(inputDataframe, variable, array, i):
    
        i = i - 1
        
        if i == 1:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i]) +     '')
            
            return df1
        if i == 2:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2
        if i == 3:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3
        if i == 4:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4
        if i == 5:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5
        if i == 6:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6
        if i == 7:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 7]) + ' and ' + str(variable) + ' < ' + str(array[i - 6]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df7 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6, df7
        if i == 8: 
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 8]) + ' and ' + str(variable) + ' < ' + str(array[i - 7]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 7]) + ' and ' + str(variable) + ' < ' + str(array[i - 6]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df7 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df8 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6, df7, df8
        if i == 9:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 9]) + ' and ' + str(variable) + ' < ' + str(array[i - 8]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 8]) + ' and ' + str(variable) + ' < ' + str(array[i - 7]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 7]) + ' and ' + str(variable) + ' < ' + str(array[i - 6]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df7 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df8 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df9 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6, df7, df8, df9
        if i == 10:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 10])+ ' and ' + str(variable) + ' < ' + str(array[i - 9]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 9]) + ' and ' + str(variable) + ' < ' + str(array[i - 8]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 8]) + ' and ' + str(variable) + ' < ' + str(array[i - 7]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 7]) + ' and ' + str(variable) + ' < ' + str(array[i - 6]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df7 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df8 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df9 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df10= inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6, df7, df8, df9, df10
        
        else:
            print('Need more entries in your array.')

In [ ]:
jets1_a ,jets2_a ,jets3_a ,jets4_a ,jets5_a ,jets6_a ,jets7_a ,jets8_a ,jets9_a ,jets10_a = bins(jets_antiKt_data,'jet_pt_antiKt',jets_pt_1,jets_pt_1.shape[0])
jets11_a,jets12_a,jets13_a,jets14_a,jets15_a,jets16_a,jets17_a,jets18_a,jets19_a,jets20_a = bins(jets_antiKt_data,'jet_pt_antiKt',jets_pt_2,jets_pt_2.shape[0])
jets21_a,jets22_a,jets23_a,jets24_a,jets25_a                                              = bins(jets_antiKt_data,'jet_pt_antiKt',jets_pt_3,jets_pt_3.shape[0])

In [ ]:
jets1_c ,jets2_c ,jets3_c ,jets4_c ,jets5_c ,jets6_c ,jets7_c ,jets8_c ,jets9_c, jets10_c = bins(jets_CamAac_data,'jet_pt_CamAac',jets_pt_1,jets_pt_1.shape[0])
jets11_c,jets12_c,jets13_c,jets14_c,jets15_c,jets16_c,jets17_c,jets18_c,jets19_c,jets20_c = bins(jets_CamAac_data,'jet_pt_CamAac',jets_pt_2,jets_pt_2.shape[0])
jets21_c,jets22_c,jets23_c,jets24_c,jets25_c                                              = bins(jets_CamAac_data,'jet_pt_CamAac',jets_pt_3,jets_pt_3.shape[0])

In [ ]:
Q1.eval('W = sqrt(W2)',inplace=True)
Q2.eval('W = sqrt(W2)',inplace=True)

In [ ]:
jets1_mult_a  = np.mean(jets1_a['jet_mult_antiKt'])
jets2_mult_a  = np.mean(jets2_a['jet_mult_antiKt'])
jets3_mult_a  = np.mean(jets3_a['jet_mult_antiKt'])
jets4_mult_a  = np.mean(jets4_a['jet_mult_antiKt'])
jets5_mult_a  = np.mean(jets5_a['jet_mult_antiKt'])
jets6_mult_a  = np.mean(jets6_a['jet_mult_antiKt'])
jets7_mult_a  = np.mean(jets7_a['jet_mult_antiKt'])
jets8_mult_a  = np.mean(jets8_a['jet_mult_antiKt'])
jets9_mult_a  = np.mean(jets9_a['jet_mult_antiKt'])

jets10_mult_a = np.mean(jets10_a['jet_mult_antiKt'])
jets11_mult_a = np.mean(jets11_a['jet_mult_antiKt'])
jets12_mult_a = np.mean(jets12_a['jet_mult_antiKt'])
jets13_mult_a = np.mean(jets13_a['jet_mult_antiKt'])
jets14_mult_a = np.mean(jets14_a['jet_mult_antiKt'])
jets15_mult_a = np.mean(jets15_a['jet_mult_antiKt'])
jets16_mult_a = np.mean(jets16_a['jet_mult_antiKt'])
jets17_mult_a = np.mean(jets17_a['jet_mult_antiKt'])
jets18_mult_a = np.mean(jets18_a['jet_mult_antiKt'])
jets19_mult_a = np.mean(jets19_a['jet_mult_antiKt'])
jets20_mult_a = np.mean(jets20_a['jet_mult_antiKt'])
jets21_mult_a = np.mean(jets21_a['jet_mult_antiKt'])
jets22_mult_a = np.mean(jets22_a['jet_mult_antiKt'])
jets23_mult_a = np.mean(jets23_a['jet_mult_antiKt'])
jets24_mult_a = np.mean(jets24_a['jet_mult_antiKt'])
jets25_mult_a = np.mean(jets25_a['jet_mult_antiKt'])

jet_mult_a = np.array([jets1_mult_a,jets2_mult_a,jets3_mult_a,jets4_mult_a,jets5_mult_a,jets6_mult_a,jets7_mult_a,
                       jets8_mult_a,jets9_mult_a,jets10_mult_a,jets11_mult_a,jets12_mult_a,jets13_mult_a,
                       jets14_mult_a,jets15_mult_a,jets16_mult_a,jets17_mult_a,jets18_mult_a,jets19_mult_a,
                       jets20_mult_a,jets21_mult_a,jets22_mult_a,jets23_mult_a,jets24_mult_a,jets25_mult_a])

jet_mult_y_err_a = np.array([np.std(jets1_a['jet_mult_antiKt']),np.std(jets2_a['jet_mult_antiKt']),np.std(jets3_a['jet_mult_antiKt']),
                           np.std(jets4_a['jet_mult_antiKt']),np.std(jets5_a['jet_mult_antiKt']),np.std(jets6_a['jet_mult_antiKt']),
                           np.std(jets7_a['jet_mult_antiKt']),np.std(jets8_a['jet_mult_antiKt']),np.std(jets9_a['jet_mult_antiKt']),
                           np.std(jets10_a['jet_mult_antiKt']),np.std(jets11_a['jet_mult_antiKt']),np.std(jets12_a['jet_mult_antiKt']),
                           np.std(jets13_a['jet_mult_antiKt']),np.std(jets14_a['jet_mult_antiKt']),np.std(jets15_a['jet_mult_antiKt']),
                           np.std(jets16_a['jet_mult_antiKt']),np.std(jets17_a['jet_mult_antiKt']),np.std(jets18_a['jet_mult_antiKt']),
                           np.std(jets19_a['jet_mult_antiKt']),np.std(jets20_a['jet_mult_antiKt']),np.std(jets21_a['jet_mult_antiKt']),
                           np.std(jets22_a['jet_mult_antiKt']),np.std(jets23_a['jet_mult_antiKt']),np.std(jets24_a['jet_mult_antiKt']),
                           np.std(jets25_a['jet_mult_antiKt'])])

In [ ]:
jets1_mult_c  = np.mean(jets1_c['jet_mult_CamAac'])
jets2_mult_c  = np.mean(jets2_c['jet_mult_CamAac'])
jets3_mult_c  = np.mean(jets3_c['jet_mult_CamAac'])
jets4_mult_c  = np.mean(jets4_c['jet_mult_CamAac'])
jets5_mult_c  = np.mean(jets5_c['jet_mult_CamAac'])
jets6_mult_c  = np.mean(jets6_c['jet_mult_CamAac'])
jets7_mult_c  = np.mean(jets7_c['jet_mult_CamAac'])
jets8_mult_c  = np.mean(jets8_c['jet_mult_CamAac'])
jets9_mult_c  = np.mean(jets9_c['jet_mult_CamAac'])

jets10_mult_c = np.mean(jets10_c['jet_mult_CamAac'])
jets11_mult_c = np.mean(jets11_c['jet_mult_CamAac'])
jets12_mult_c = np.mean(jets12_c['jet_mult_CamAac'])
jets13_mult_c = np.mean(jets13_c['jet_mult_CamAac'])
jets14_mult_c = np.mean(jets14_c['jet_mult_CamAac'])
jets15_mult_c = np.mean(jets15_c['jet_mult_CamAac'])
jets16_mult_c = np.mean(jets16_c['jet_mult_CamAac'])
jets17_mult_c = np.mean(jets17_c['jet_mult_CamAac'])
jets18_mult_c = np.mean(jets18_c['jet_mult_CamAac'])
jets19_mult_c = np.mean(jets19_c['jet_mult_CamAac'])
jets20_mult_c = np.mean(jets20_c['jet_mult_CamAac'])
jets21_mult_c = np.mean(jets21_c['jet_mult_CamAac'])
jets22_mult_c = np.mean(jets22_c['jet_mult_CamAac'])
jets23_mult_c = np.mean(jets23_c['jet_mult_CamAac'])
jets24_mult_c = np.mean(jets24_c['jet_mult_CamAac'])
jets25_mult_c = np.mean(jets25_c['jet_mult_CamAac'])

jet_mult_c = np.array([jets1_mult_c,jets2_mult_c,jets3_mult_c,jets4_mult_c,jets5_mult_c,jets6_mult_c,jets7_mult_c,
                       jets8_mult_c,jets9_mult_c,jets10_mult_c,jets11_mult_c,jets12_mult_c,jets13_mult_c,
                       jets14_mult_c,jets15_mult_c,jets16_mult_c,jets17_mult_c,jets18_mult_c,jets19_mult_c,
                       jets20_mult_c,jets21_mult_c,jets22_mult_c,jets23_mult_c,jets24_mult_c,jets25_mult_c])

jet_mult_y_err_c = np.array([np.std(jets1_c['jet_mult_CamAac']),np.std(jets2_c['jet_mult_CamAac']),np.std(jets3_c['jet_mult_CamAac']),
                           np.std(jets4_c['jet_mult_CamAac']),np.std(jets5_c['jet_mult_CamAac']),np.std(jets6_c['jet_mult_CamAac']),
                           np.std(jets7_c['jet_mult_CamAac']),np.std(jets8_c['jet_mult_CamAac']),np.std(jets9_c['jet_mult_CamAac']),
                           np.std(jets10_c['jet_mult_CamAac']),np.std(jets11_c['jet_mult_CamAac']),np.std(jets12_c['jet_mult_CamAac']),
                           np.std(jets13_c['jet_mult_CamAac']),np.std(jets14_c['jet_mult_CamAac']),np.std(jets15_c['jet_mult_CamAac']),
                           np.std(jets16_c['jet_mult_CamAac']),np.std(jets17_c['jet_mult_CamAac']),np.std(jets18_c['jet_mult_CamAac']),
                           np.std(jets19_c['jet_mult_CamAac']),np.std(jets20_c['jet_mult_CamAac']),np.std(jets21_c['jet_mult_CamAac']),
                           np.std(jets22_c['jet_mult_CamAac']),np.std(jets23_c['jet_mult_CamAac']),np.std(jets24_c['jet_mult_CamAac']),
                           np.std(jets25_c['jet_mult_CamAac'])])

In [ ]:
cross_section = 6.959e-4

a, b, _ = plt.hist(Q1['eOut_pt'],bins=60)
a_new   = a*((1e6 * cross_section) / numEvents)

c, d, _ = plt.hist(Q2['eOut_pt'],bins=60)
c_new   = c*((1e6 * cross_section) / numEvents)

e, f, _ = plt.hist(Q1['W'],bins=30)
e_new   = e*((1e6 * cross_section) / numEvents)

g, h, _ = plt.hist(Q2['W'],bins=30)
g_new   = g*((1e6 * cross_section) / numEvents)

i, j, _ = plt.hist(Q1['x_bjork'],bins=200)
i_new   = i*((1e6 * cross_section) / numEvents)

k, l, _ = plt.hist(Q2['x_bjork'],bins=200)
k_new   = k*((1e6 * cross_section) / numEvents)

m, n, _ = plt.hist(Q1['y'],bins=60)
m_new   = m*((1e6 * cross_section) / numEvents)

o, p, _ = plt.hist(Q2['y'],bins=60)
o_new   = o*((1e6 * cross_section) / numEvents)

q, r, _ = plt.hist(Q1['nu'],bins=45)
q_new   = q*((1e6 * cross_section) / numEvents)

s, t, _ = plt.hist(Q2['nu'],bins=45)
s_new   = s*((1e6 * cross_section) / numEvents)

u, v, _ = plt.hist(Q1['phi_corr'],bins=40)
u_new   = u*((1e6 * cross_section) / numEvents)

w, x, _ = plt.hist(Q2['phi_corr'],bins =40)
w_new   = w*((1e6 * cross_section) / numEvents)

In [ ]:
fig = plt.figure(figsize=(40,55))

plt.subplot(6,2,1)
plt.bar(b[0:60],a_new,color='red')
plt.plot([],[],'',label=r'$\mathrm{Q}^2 = 1-10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\mathrm{lepton} \:\: p_\mathrm{T} \:\: (\mathrm{GeV}/c)$', fontsize = 35)
plt.xticks(fontsize = 20)
plt.xlim(0,20)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{d} p_\mathrm{T}} \:\: (1/\mathrm{GeV})$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,2)
plt.bar(d[0:60],c_new,color='red')
plt.xlabel(r'$\mathrm{lepton} \:\: p_\mathrm{T} \:\: (\mathrm{GeV}/c)$', fontsize = 35)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 > 10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xticks(fontsize = 20)
plt.xlim(0,20)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{d} p_\mathrm{T}} \:\: (1/\mathrm{GeV})$', fontsize = 44)
plt.yticks(fontsize =20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

# plt.subplot(5,2,3)
# plt.hist(Q1['jet_pt'],bins=60,histtype='step',color='red')
# plt.xlabel(r'$\mathrm{jet} \:\: p_\mathrm{T} \:\: (\mathrm{GeV}/c)$', fontsize = 35)
# plt.xticks(fontsize = 18)
# plt.ylabel(r'$\mathrm{Counts}$', fontsize = 34)
# plt.yticks(fontsize =18)
# plt.legend(loc='upper right', frameon=False, prop={'size':26})

# plt.subplot(5,2,4)
# plt.hist(Q2['jet_pt'],bins=60,histtype='step',color='red')
# plt.xlabel(r'$\mathrm{jet} \:\: p_\mathrm{T} \:\: (\mathrm{GeV}/c)$', fontsize = 35)
# plt.xticks(fontsize = 18)
# plt.ylabel(r'$\mathrm{Counts}$', fontsize = 34)
# plt.yticks(fontsize =18)
# plt.legend(loc='upper right', frameon=False, prop={'size':26})

plt.subplot(6,2,3)
plt.bar(f[0:30],e_new,color='blue',width=4.2)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 = 1-10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\mathrm{W} \:\: (\mathrm{GeV}/c^2)$', fontsize = 35)
plt.xticks(fontsize = 20)
plt.xlim(0,140)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{d} \mathrm{W}} \:\: (1/\mathrm{GeV})$', fontsize = 44)
plt.yticks(fontsize =20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,4)
plt.bar(h[0:30],g_new,color='blue',width=4.5)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 > 10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\mathrm{W} \:\: (\mathrm{GeV}/c^2)$', fontsize = 35)
plt.xticks(fontsize = 20)
plt.xlim(0,140)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{d} \mathrm{W}} \:\: (1/\mathrm{GeV})$', fontsize = 44)
plt.yticks(fontsize =20)
plt.ylim(0,2)
plt.legend(loc='upper right', frameon=False, prop={'size': 31})

plt.subplot(6,2,5)
plt.bar(j[0:200],i_new,color='green',width=0.00022)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 = 1-10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\mathrm{x_{bjork}}$', fontsize = 35)
plt.xticks(fontsize = 20)
# plt.xlim(1e-4,1)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{dx_{bjork}}}$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.xscale('log')
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,6)
plt.bar(l[0:200],k_new,color='green',width=0.0039)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 > 10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\mathrm{x_{bjork}}$', fontsize = 35)
plt.xticks(fontsize = 20)
# plt.xlim(1e-4,1)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{dx_{bjork}}}$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.xscale('log')
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,7)
plt.bar(n[0:60],m_new,color='orange',width=0.018)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 = 1-10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\mathrm{y}$', fontsize = 35)
plt.xticks(fontsize = 20)
plt.xlim(0,1)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{dy}}$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,8)
plt.bar(p[0:60],o_new,color='orange',width=0.02)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 > 10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\mathrm{y}$', fontsize = 35)
plt.xticks(fontsize = 20)
plt.xlim(0,1)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{dy}}$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,9)
plt.bar(r[0:45],q_new,color='purple',width=250)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 = 1-10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\nu \:\: (\mathrm{GeV})$', fontsize = 35)
plt.xticks(fontsize = 20)
plt.xlim(0,10700)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{d} \nu} \:\: (1 / \mathrm{GeV})$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,10)
plt.bar(t[0:45],s_new,color='purple',width=250)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 > 10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.xlabel(r'$\nu \:\: (\mathrm{GeV})$', fontsize = 35)
plt.xticks(fontsize = 20)
plt.xlim(0,10700)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{d} \nu} \:\: (1 / \mathrm{GeV})$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,11)
plt.bar(v[0:40],u_new,color='red',width=0.08)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 = 1-10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{anti}-k_{\mathrm{T}} \:\: R=1.0$', color='white')
plt.plot([],[],'',label=r'$p_{\mathrm{T}}^{\mathrm{jet}} \: > \: 5 \:\: \mathrm{GeV}/c$', color='white')
plt.xlabel(r'$\Delta \phi \:\: (\mathrm{rads.})$', fontsize = 35)
plt.xlim(0,np.pi)
plt.xticks(fontsize = 20)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{d} \Delta \phi} \:\: (1 / \mathrm{rads.})$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

plt.subplot(6,2,12)
plt.bar(x[0:40],w_new,color='red',width=0.08)
plt.plot([],[],'',label=r'$\mathrm{Q}^2 > 10 \:\: \mathrm{GeV}^{2}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{anti}-k_{\mathrm{T}} \:\: R=1.0$', color='white')
plt.plot([],[],'',label=r'$p_{\mathrm{T}}^{\mathrm{jet}} \: > \: 5 \:\: \mathrm{GeV}/c$', color='white')
plt.xlabel(r'$\Delta \phi \:\: (\mathrm{rads.})$', fontsize = 35)
plt.xlim(0,np.pi)
plt.xticks(fontsize = 20)
plt.ylabel(r'$\frac{\mathrm{d \sigma}}{\mathrm{d} \Delta \phi} \:\: (1 / \mathrm{rads.})$', fontsize = 44)
plt.yticks(fontsize = 20)
plt.legend(loc='upper right', frameon=False, prop={'size':31})

# plt.subplot(5,2,8)
# plt.hist(part_data['eta_corr'],bins =50,histtype='step',color='red')
# plt.xlabel(r'$\Delta \eta$', fontsize = 35)
# plt.xticks(fontsize = 18)
# plt.ylabel(r'$\mathrm{Counts}$', fontsize = 34)
# plt.yticks(fontsize =18)
# plt.legend(loc='upper right', frameon=False, prop={'size':26})

# plt.savefig('ep_sim_distributions.pdf')

In [ ]:
fig = plt.figure(figsize=(8,5))

plt.errorbar(jet_mult_x, jet_mult_a, xerr=jet_mult_x_err, yerr=jet_mult_y_err_a, fmt='o',label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.errorbar(jet_mult_x, jet_mult_c, xerr=jet_mult_x_err, yerr=jet_mult_y_err_c, fmt='o', color = 'orange',label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: p_\mathrm{T} \:\: (\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,30)
plt.ylabel(r'$\mathrm{Number \:\: of \:\: Constituents}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.ylim(0,30)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

In [ ]:
plt.hist(jets1_a['jet_mult_antiKt'],bins = 5)
print np.mean(jets1_a['jet_mult_antiKt'])
print np.std(jets1_a['jet_mult_antiKt'])
print jet_mult_y_err_a[0]
print jets1_a['jet_mult_antiKt'].shape[0]
print jets1_a.query('jet_mult_antiKt > 4 and jet_mult_antiKt < 10').shape[0] / float(jets1_a['jet_mult_antiKt'].shape[0])

In [ ]:
#Q2 bins for jet distributions

jets_antiKt_data_1_3  = jets_antiKt_data.query('Q2 < 3')
jets_antiKt_data_3_10 = jets_antiKt_data.query('3 < Q2 and Q2 < 10')
jets_antiKt_data_10   = jets_antiKt_data.query('10 < Q2')

jets_CamAac_data_1_3  = jets_CamAac_data.query('Q2 < 3')
jets_CamAac_data_3_10 = jets_CamAac_data.query('3 < Q2 and Q2 < 10')
jets_CamAac_data_10   = jets_CamAac_data.query('10 < Q2')

In [ ]:
fig = plt.figure(figsize=(18,20))

plt.subplot(3,2,1)
plt.hist(jets_antiKt_data_1_3['jet_pt_antiKt'],bins = 45,histtype = 'step',label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_1_3['jet_pt_CamAac'],bins = 45,histtype = 'step',label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.plot([],[],'',label=r'$ 1 \: < \: \mathrm{Q}^2 \: < \: 3 \: \mathrm{GeV}^2$', color='white' )
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{p}_{T} \:\:(\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,40)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,2)
plt.hist(jets_antiKt_data_1_3['jet_mult_antiKt'],bins = 23,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_1_3['jet_mult_CamAac'],bins = 24,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{multiplicity}$', fontsize = 25)
plt.xticks(fontsize = 17)
# plt.xlim(0,40)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,3)
plt.hist(jets_antiKt_data_1_3['jet_phi_antiKt'],bins = 40,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_1_3['jet_phi_CamAac'],bins = 40,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \phi \:\:(\mathrm{rads.})$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
# plt.ylim(0,380)
plt.legend(loc='upper left', frameon=False, prop={'size':15})

plt.subplot(3,2,4)
plt.hist(jets_antiKt_data_1_3['jet_eta_antiKt'],bins = 25,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_1_3['jet_eta_CamAac'],bins = 25,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \eta$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
# plt.ylim(0,500)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,5)
plt.hist(jets_antiKt_data_1_3['jet_mass_antiKt'],bins = 30,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_1_3['jet_mass_CamAac'],bins = 30,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{mass} \:\:(\mathrm{GeV})$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

print np.mean(jets_antiKt_data_1_3['jet_mult_antiKt'])
print np.mean(jets_antiKt_data_1_3['jet_eta_antiKt'])

# plt.savefig('ep_sim_jet_distributions.pdf')

In [ ]:
fig = plt.figure(figsize=(18,20))

plt.subplot(3,2,1)
plt.hist(jets_antiKt_data_3_10['jet_pt_antiKt'],bins = 45,histtype = 'step',label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_3_10['jet_pt_CamAac'],bins = 45,histtype = 'step',label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.plot([],[],'',label=r'$ 3 \: < \: \mathrm{Q}^2 \: < \: 10 \: \mathrm{GeV}^2$', color='white' )
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{p}_{T} \:\:(\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,40)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,2)
plt.hist(jets_antiKt_data_3_10['jet_mult_antiKt'],bins = 29,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_3_10['jet_mult_CamAac'],bins = 28,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{multiplicity}$', fontsize = 25)
plt.xticks(fontsize = 17)
# plt.xlim(0,40)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,3)
plt.hist(jets_antiKt_data_3_10['jet_phi_antiKt'],bins = 40,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_3_10['jet_phi_CamAac'],bins = 40,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \phi \:\:(\mathrm{rads.})$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
# plt.ylim(0,380)
plt.legend(loc='upper left', frameon=False, prop={'size':15})

plt.subplot(3,2,4)
plt.hist(jets_antiKt_data_3_10['jet_eta_antiKt'],bins = 18,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_3_10['jet_eta_CamAac'],bins = 18,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \eta$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
# plt.ylim(0,500)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,5)
plt.hist(jets_antiKt_data_3_10['jet_mass_antiKt'],bins = 30,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_3_10['jet_mass_CamAac'],bins = 30,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{mass} \:\:(\mathrm{GeV})$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

print np.mean(jets_antiKt_data_3_10['jet_mult_antiKt'])
print np.mean(jets_antiKt_data_3_10['jet_eta_antiKt'])

# plt.savefig('ep_sim_jet_distributions.pdf')

In [ ]:
fig = plt.figure(figsize=(18,20))

plt.subplot(3,2,1)
plt.hist(jets_antiKt_data_10['jet_pt_antiKt'],bins = 45,histtype = 'step',label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_10['jet_pt_CamAac'],bins = 45,histtype = 'step',label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.plot([],[],'',label=r'$\mathrm{Q}^2 \: > \: 10 \: \mathrm{GeV}^2$', color='white' )
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{p}_{T} \:\:(\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,40)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,2)
plt.hist(jets_antiKt_data_10['jet_mult_antiKt'],bins = 30,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_10['jet_mult_CamAac'],bins = 29,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{multiplicity}$', fontsize = 25)
plt.xticks(fontsize = 17)
# plt.xlim(0,40)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,3)
plt.hist(jets_antiKt_data_10['jet_phi_antiKt'],bins = 40,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_10['jet_phi_CamAac'],bins = 40,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \phi \:\:(\mathrm{rads.})$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
# plt.ylim(0,380)
plt.legend(loc='upper left', frameon=False, prop={'size':15})

plt.subplot(3,2,4)
plt.hist(jets_antiKt_data_10['jet_eta_antiKt'],bins = 20,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_10['jet_eta_CamAac'],bins = 20,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \eta$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
# plt.ylim(0,500)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(3,2,5)
plt.hist(jets_antiKt_data_10['jet_mass_antiKt'],bins = 30,histtype = 'step') #,label=r'$\mathrm{anti}-k_{\mathrm{T}}, \: R=1.0$')
plt.hist(jets_CamAac_data_10['jet_mass_CamAac'],bins = 30,histtype = 'step') #,label=r'$\mathrm{Cambridge-Aachen}, \: R=1.0$')
plt.xlabel(r'$\mathrm{jet} \:\: \mathrm{mass} \:\:(\mathrm{GeV})$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.ylabel(r'$\mathrm{Counts}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

print np.mean(jets_antiKt_data_10['jet_mult_antiKt'])
print np.mean(jets_antiKt_data_10['jet_eta_antiKt'])

# plt.savefig('ep_sim_jet_distributions.pdf')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

# setup the figure and axes
fig = plt.figure(figsize=(8, 8))
ax1 = fig.add_subplot(111, projection='3d')

# fake data
x = jets_antiKt_data['jet_eta_antiKt']
y = jets_antiKt_data['jet_phi_antiKt']
z = [0]*(len(y))

dx = [0.1]*len(y)
dy = [0.1]*len(y)
dz = jets_antiKt_data['jet_pt_antiKt']


ax1.bar3d(x, y, z, dx, dy, dz, zsort = 'average', shade=True)

ax1.view_init(elev = 15, azim = 25)
ax1.set_xlabel(r'$\eta$', fontsize = 14)
ax1.set_ylabel(r'$\phi \:\: (rad.)$', fontsize = 14)
ax1.set_zlabel(r'jet $\mathrm{p}_{T} \:\: (\mathrm{GeV}/c)$', fontsize = 15)
ax1.tick_params(axis='both', which='major', labelsize=8)

# plt.savefig('ep_sim_jet_distribution_50Events.pdf')

plt.show()

In [ ]:
fig = plt.figure(figsize=(8,5))

plt.hist(jets_antiKt_data['angularity_0'],bins = 40,histtype='step',label=r'$\mathrm{a = 0}$');
plt.hist(jets_antiKt_data['angularity_1'],bins = 40,histtype='step',label=r'$\mathrm{a = 1}$');
plt.plot([],[],'',label=r'$\mathrm{anti}-k_{\mathrm{T}}, \:\: R=1.0$', color='white')
plt.xlabel(r'$\tau_a$', fontsize = 25)
plt.xlim(0,1)
plt.xticks(fontsize=17)
plt.ylabel('Counts',fontsize=20)
plt.yticks(fontsize=17)
plt.legend(prop={'size': 17},frameon=False)
plt.legend(loc='upper right', frameon=False, prop={'size':15})

In [ ]:
xmin = np.amin(part_data['x_bjork'])
xmax = np.amax(part_data['x_bjork'])
ymin = np.amin(part_data['Q2'])
ymax = np.amax(part_data['Q2'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(part_data['x_bjork'], part_data['Q2'], bins='log',xscale='log',yscale='log', gridsize=125, cmap='inferno')
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_xlabel(r'$\mathrm{x}_{\mathrm{bjork}}$', fontsize = 20)
ax.set_ylabel('$\mathrm{Q}^2 \:\: (\mathrm{GeV}^2)$', fontsize=20)
cb = fig.colorbar(hb, ax=ax)
cb.set_label('Counts', fontsize=20)

plt.show()

In [ ]:
xmin = np.amin(part_data['eOut_pt'])
xmax = np.amax(part_data['eOut_pt'])
ymin = np.amin(part_data['eOut_eta'])
ymax = np.amax(part_data['eOut_eta'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(part_data['eOut_pt'], part_data['eOut_eta'], bins='log',xscale='log', gridsize=125, cmap='inferno')
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_xlabel(r'$\mathrm{p}_{T} (\mathrm{GeV}/c)$', fontsize = 20)
ax.set_ylabel('$\eta$', fontsize=20)
cb = fig.colorbar(hb, ax=ax)
cb.set_label('Counts', fontsize=20)

plt.show()

In [ ]:
# part_data.to_csv('particle_data.csv', sep=',')
# jets_data.to_csv('jets_data.csv', sep=',')

In [ ]:
# part_data.plot.hexbin(x='x_bjork', y='Q2', bins='log',xscale='log',yscale='log', gridsize=50, cmap='inferno')

In [ ]:
#how I was doing it before

# arrayArb = [0]*event.all().shape[0]

# def getData(array, variable):
#     for i in range(0, event.all().shape[0]):
#         array[i] = event.all()[i][variable]
#     return array

# pxArray = getData(arrayArb, 1)
# pyArray = getData(arrayArb, 2)

# ptArray = np.sqrt(np.power(pxArray,2) + np.power(pyArray,2))

In [ ]:
# numEvents = 3e4

# j = 0
# k = 0
# l = 0
# m = 0
# n = 0
# p = 0
# q = 0
# t = 0
# z = 0

# ################################

# p_mass    = 0

# eIn_pt    = [0]*1000000
# eIn_p     = [0]*1000000

# eIn_4vec  = ROOT.TLorentzVector()

# p_pt      = [0]*1000000
# p_p       = [0]*1000000

# p_4vec    = ROOT.TLorentzVector()

# eOut_pt   = [0]*1000000
# eOut_p    = [0]*1000000
# eOut_eta  = [0]*1000000

# eOut_4vec = ROOT.TLorentzVector()

# Q2        = [0]*1000000
# W2        = [0]*1000000
# x_bjork   = [0]*1000000
# y         = [0]*1000000
# nu        = [0]*1000000

# ################################

# Q2_jets_antiKt = [0]*1000000
# Q2_jets_CamAac = [0]*1000000

# ################################

# jet_pt_antiKt   = [0]*1000000
# jet_eta_antiKt  = [0]*1000000
# jet_phi_antiKt  = [0]*1000000
# jet_mass_antiKt = [0]*1000000
# jet_mult_antiKt = [0]*1000000

# jet_pt_CamAac   = [0]*1000000
# jet_eta_CamAac  = [0]*1000000
# jet_phi_CamAac  = [0]*1000000
# jet_mass_CamAac = [0]*1000000
# jet_mult_CamAac = [0]*1000000

# phi_corr = [0]*1000000
# eta_corr = [0]*1000000

# ################################

# R_0 = [0]*1000000
# R_1 = [0]*1000000

# angularity_0 = [0]*1000000
# angularity_1 = [0]*1000000

# ################################

# for event in pythia(events=numEvents): #hepmc_read('events.hepmc'):
    
#     if not event: continue
    
#     jets_antiKt = cluster(event.all(selection & (ABS_PDG_ID != 11)), R=1, p=-1, ep=True).inclusive_jets(ptmin=5)
#     jets_CamAac = cluster(event.all(selection & (ABS_PDG_ID != 11)), R=1, p= 0, ep=True).inclusive_jets(ptmin=5)

#     if(len(jets_antiKt)<1): continue

# ################################

#     for i in event.all():
        
#         if i['pdgid'] == 11 and i['status'] == 4:
            
#             #mass      = ROOT.TDatabasePDG.Instance().GetParticle(11).Mass()
#             eIn_pt[j] = np.sqrt(i['px']*i['px'] + i['py']*i['py'])
#             eIn_p[j]  = np.sqrt(i['px']*i['px'] + i['py']*i['py'] + i['pz']*i['pz'])
            
#             #make its respective 4 vector which can be used, along with the others, to find Q2, W, x_borj, and y

#             eIn_4vec.SetPxPyPzE(i['px'], i['py'], i['pz'], i['E'])
            
#             j = j  + 1
            
#         elif i['pdgid'] == 2212 and i['status'] == 4:
            
#             p_pt[k] = np.sqrt(i['px']*i['px'] + i['py']*i['py'])
#             p_p[k]  = np.sqrt(i['px']*i['px'] + i['py']*i['py'] + i['pz']*i['pz'])
            
#             p_mass  = i['mass']
            
#             p_4vec.SetPxPyPzE(i['px'], i['py'], i['pz'], i['E'])
        
#             k = k  + 1
            
#         elif i['pdgid'] == 11 and i['status'] == 1 and i['prodt'] == 0:
            
#             eOut_pt[l]  = np.sqrt(i['px']*i['px'] + i['py']*i['py'])
#             eOut_p[l]   = np.sqrt(i['px']*i['px'] + i['py']*i['py'] + i['pz']*i['pz'])
#             eOut_eta[l] = 0.5*np.log((eOut_p[l] + i['pz'])/(eOut_p[l] - i['pz']))
            
#             eOut_4vec.SetPxPyPzE(i['px'], i['py'], i['pz'], i['E'])
            
#             l = l  + 1
            
#     def dot(a_4vec, b_4vec):
        
#         num = a_4vec(3)*b_4vec(3) - a_4vec(0)*b_4vec(0) - a_4vec(1)*b_4vec(1) - a_4vec(2)*b_4vec(2)
        
#         return num
        
#     gm_4vec = eIn_4vec - eOut_4vec
    
#     Q2[m]      = -gm_4vec.Mag2()
#     W2[m]      = (p_4vec + gm_4vec).Mag2()
#     x_bjork[m] = Q2[m] / (2*dot(p_4vec,gm_4vec))
#     y[m]       = (dot(p_4vec,gm_4vec)) / (dot(p_4vec,eIn_4vec))
#     nu[m]      = (dot(p_4vec,gm_4vec)) / (p_mass)
        
#     m = m + 1

# ################################

#     for jet in jets_antiKt:
        
#         jet_pt_antiKt[n]   = jet.pt
#         jet_eta_antiKt[n]  = jet.eta
#         jet_phi_antiKt[n]  = jet.phi
#         jet_mass_antiKt[n] = jet.mass
#         jet_mult_antiKt[n] = len(jet)
        
#         phi_corr[n] = np.abs(ROOT.TVector2.Phi_mpi_pi(jet.phi-gm_4vec.Phi()))
        
#         eta_corr[n] = np.abs(jet.eta-gm_4vec.Eta())

#         ###angularity
#         for part in jet:

#             R_0[p] += part.pt * ((ROOT.TVector2.Phi_mpi_pi(jet_phi_antiKt[n]-part.phi))**2 + (jet_eta_antiKt[n]-part.eta)**2)
#             R_1[p] += part.pt * np.sqrt((ROOT.TVector2.Phi_mpi_pi(jet_phi_antiKt[n]-part.phi))**2 + (jet_eta_antiKt[n]-part.eta)**2)
            
#         angularity_0[p] = (1 / jet_pt_antiKt[n]) * R_0[p]
#         angularity_1[p] = (1 / jet_pt_antiKt[n]) * R_1[p]

#         n = n + 1
#         p = p + 1
        
# ################################
        
#     r_1 = 0
#     r_2 = 0
    
#     while r_2 < (r_1 + len(jets_antiKt)):
#         Q2_jets_antiKt[t] = Q2[m - 1]
#         r_2 = r_2 + 1
#         t = t + 1   
        
# ################################
        
#     if(len(jets_CamAac)<1): continue
        
#     for jet in jets_CamAac:
        
#         jet_pt_CamAac[q]   = jet.pt
#         jet_eta_CamAac[q]  = jet.eta
#         jet_phi_CamAac[q]  = jet.phi
#         jet_mass_CamAac[q] = jet.mass
#         jet_mult_CamAac[q] = len(jet)
        
#         q = q + 1
        
# ################################
        
#     x_1 = 0
#     x_2 = 0
    
#     while x_2 < (x_1 + len(jets_CamAac)):
#         Q2_jets_CamAac[z] = Q2[m - 1]
#         x_2 = x_2 + 1
#         z = z + 1   
        
# ################################
        
# part_dataframe        = pd.DataFrame({'eIn_pt': eIn_pt, 'eOut_pt': eOut_pt, 'eIn_p': eIn_p, 'eOut_p': eOut_p, 
#                                'eOut_eta': eOut_eta, 'p_pt': p_pt, 'p_p': p_p, 'Q2': Q2, 'W2': W2, 'x_bjork': x_bjork,
#                                'y': y, 'nu': nu, 'phi_corr': phi_corr , 'eta_corr': eta_corr})
# part_data             = part_dataframe.query('eIn_pt != 0 and p_pt != 0 and y < 0.95')

# jets_antiKt_dataframe = pd.DataFrame({'jet_pt_antiKt': jet_pt_antiKt, 'jet_eta_antiKt': jet_eta_antiKt, 
#                                       'jet_phi_antiKt':jet_phi_antiKt,'jet_mass_antiKt': jet_mass_antiKt, 
#                                       'jet_mult_antiKt':jet_mult_antiKt,'angularity_0': angularity_0,   
#                                       'angularity_1': angularity_1, 'Q2': Q2_jets_antiKt})
# jets_antiKt_data      = jets_antiKt_dataframe.query('jet_pt_antiKt != 0')

# jets_CamAac_dataframe = pd.DataFrame({'jet_pt_CamAac': jet_pt_CamAac,  'jet_eta_CamAac':jet_eta_CamAac,
#                                       'jet_phi_CamAac': jet_phi_CamAac,'jet_mass_CamAac':jet_mass_CamAac,
#                                       'jet_mult_CamAac': jet_mult_CamAac, 'Q2': Q2_jets_CamAac})
# jets_CamAac_data      = jets_CamAac_dataframe.query('jet_pt_CamAac != 0')